In [420]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/homework-1-dataset/sample_submission.csv
/kaggle/input/homework-1-dataset/data_description.txt
/kaggle/input/homework-1-dataset/train.csv
/kaggle/input/homework-1-dataset/test.csv


In [421]:
df = pd.read_csv('/kaggle/input/homework-1-dataset/train.csv')

# Train/Test Split and Feature Engineering

In [422]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

X_train, X_test = train_test_split(df,test_size=0.2, random_state=42)

# dropping null values
null_ratio = X_train.isna().mean()
too_much_null_columns = null_ratio[null_ratio >= 0.5]
X_train.drop(columns = too_much_null_columns.index, inplace = True)
X_test.drop(columns = too_much_null_columns.index, inplace = True)

# filling numbers with average value
for col in X_train.select_dtypes(include=['number']):
    change_null_to_this = X_train[col].mean()
    X_train.fillna(change_null_to_this, inplace=True)
    X_test.fillna(change_null_to_this, inplace=True)

# filling categoricals with mode value
for col in X_train.select_dtypes(include=['object']):
    change_null_to_this = X_train[col].mode()
    X_train.fillna(change_null_to_this, inplace=True)
    X_test.fillna(change_null_to_this, inplace=True)

# handling categorical values
for col in X_train.select_dtypes(include=['object']):
    encoder = OrdinalEncoder()
    
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)
    
    X_train[col] = encoder.fit_transform(X_train[[col]])
    X_test[col] = encoder.fit_transform(X_test[[col]])

In [423]:
X_train.select_dtypes(include=['object']).shape, X_test.select_dtypes(include=['object']).shape

((1168, 0), (292, 0))

# Normal Training

In [424]:
Y_train = X_train.pop('SalePrice')
Y_test = X_test.pop('SalePrice')

In [425]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((1168, 75), (1168,), (292, 75), (292,))

In [1]:
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(
    max_depth=10,
    min_samples_split=25,
    min_samples_leaf=7,
    random_state=42
)
model.fit(X_train, Y_train)

prob_train = model.predict(X_train)
prob_test = model.predict(X_test)

NameError: name 'X_train' is not defined

In [ ]:
from sklearn.metrics import *

print("Train:")
r2 = r2_score(Y_train, prob_train)
print(f"R2 score: {r2}")
rmse = mean_squared_error(Y_train, prob_train, squared=False)
print(f"RMSE score: {rmse}")
mae = mean_absolute_error(Y_train, prob_train)
print(f"MAE score: {mae}")

print()
print("Test: ")

r2 = r2_score(Y_test, prob_test)
print(f"R2 score: {r2}")
rmse = mean_squared_error(Y_test, prob_test, squared=False)
print(f"RMSE score: {rmse}")
mae = mean_absolute_error(Y_test, prob_test)
print(f"MAE score: {mae}")

# using k-fold cross validation

In [428]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DecisionTreeRegressor(random_state=42))
])

scalers = [
    StandardScaler(),
    None
]

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    'regressor__max_depth': [5, 10, None],
    'regressor__min_samples_split': [5, 15, 25],
    'regressor__min_samples_leaf': [3, 5, 7]
}

In [429]:
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=kfold,
    scoring='neg_root_mean_squared_error',
    verbose=2,
    return_train_score=True
)

In [430]:
grid_search.fit(X_train, Y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=5; total time=   0.1s
[CV] END regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=5; total time=   0.1s
[CV] END regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=5; total time=   0.1s
[CV] END regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=5; total time=   0.1s
[CV] END regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=5; total time=   0.1s
[CV] END regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=15; total time=   0.1s
[CV] END regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=15; total time=   0.1s
[CV] END regressor__max_depth=5, regressor__min_samples_leaf=3, regressor__min_samples_split=15; total time=   0.1s
[CV] END regres

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'Ho...
       'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object'))])),
                                       ('regressor',
                                        DecisionTreeRegressor(random_state=42))]),
             param_grid={'regressor__max_depth': [5, 10, None],
                         'regressor__min_samples_leaf': [3, 5, 7],
                         'regressor__min_samples_split': [5, 15, 25]},
             return_train_score=True, scoring='neg_root_mean_squared_error',
             verbose=2)

In [431]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__max_depth,param_regressor__min_samples_leaf,param_regressor__min_samples_split,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
1,0.049152,0.002333,0.017111,0.000508,5,3,15,"{'regressor__max_depth': 5, 'regressor__min_sa...",-41253.686996,-45133.939324,...,-42276.719773,3215.551176,27,-29120.277675,-28769.908043,-29788.683770,-30587.732838,-30109.798514,-29675.280168,657.565646
18,0.092425,0.002283,0.017687,0.001054,None,3,5,"{'regressor__max_depth': None, 'regressor__min...",-39304.424299,-41536.070580,...,-42077.634432,4129.707304,26,-15010.464388,-14660.050304,-13136.978644,-13338.829482,-15346.651377,-14298.594839,895.134910
0,0.051052,0.001637,0.018235,0.000777,5,3,5,"{'regressor__max_depth': 5, 'regressor__min_sa...",-40954.609922,-44543.050133,...,-42031.381539,3658.434881,25,-28801.999561,-28113.215593,-27770.060820,-29855.112586,-28728.567182,-28653.791148,713.292592
2,0.047379,0.001236,0.017151,0.000403,5,3,25,"{'regressor__max_depth': 5, 'regressor__min_sa...",-40259.283289,-44191.261325,...,-41889.851888,2512.949231,24,-30919.518960,-31001.764501,-30683.927525,-30587.732838,-30910.966885,-30820.782142,157.275571
9,0.081217,0.001494,0.018068,0.001426,10,3,5,"{'regressor__max_depth': 10, 'regressor__min_s...",-38245.182381,-41152.154984,...,-41393.227483,4188.278290,23,-16266.375868,-15838.546343,-14029.479257,-14182.427713,-16373.064495,-15337.978735,1022.874454
4,0.048313,0.001670,0.017433,0.000968,5,5,15,"{'regressor__max_depth': 5, 'regressor__min_sa...",-39526.283035,-43726.668872,...,-40668.316812,3675.027918,22,-29623.125185,-29615.335797,-29789.678619,-31065.220822,-31062.411500,-30231.154384,682.713425
19,0.071632,0.001734,0.017283,0.000459,None,3,15,"{'regressor__max_depth': None, 'regressor__min...",-38912.079145,-41894.923817,...,-40578.787653,3316.041969,21,-19805.889374,-19076.228914,-22355.384286,-19467.350611,-22182.415154,-20577.453668,1401.303299
10,0.067876,0.001839,0.016328,0.000102,10,3,15,"{'regressor__max_depth': 10, 'regressor__min_s...",-38416.765152,-42061.556017,...,-40552.258797,3248.522467,20,-20401.560015,-19457.145643,-22548.935081,-19676.677661,-22470.557748,-20910.975229,1342.521875
3,0.048313,0.001535,0.017173,0.000602,5,5,5,"{'regressor__max_depth': 5, 'regressor__min_sa...",-40060.667923,-42939.255713,...,-40424.793893,3894.718597,19,-29513.225439,-29175.607244,-28873.325147,-30866.298737,-29725.477524,-29630.786818,682.673178
5,0.049096,0.002664,0.017779,0.001127,5,5,25,"{'regressor__max_depth': 5, 'regressor__min_sa...",-39646.486977,-40511.094180,...,-40256.902528,2784.537852,18,-31120.659843,-31880.206484,-30683.927525,-32236.033286,-31762.196372,-31536.604702,558.251310


In [432]:
results = pd.DataFrame(grid_search.cv_results_)
results = results.sort_values('mean_test_score')

In [433]:
print(results[[
    'mean_train_score',
    'mean_test_score'
]])


    mean_train_score  mean_test_score
1      -29675.280168    -42276.719773
18     -14298.594839    -42077.634432
0      -28653.791148    -42031.381539
2      -30820.782142    -41889.851888
9      -15337.978735    -41393.227483
4      -30231.154384    -40668.316812
19     -20577.453668    -40578.787653
10     -20910.975229    -40552.258797
3      -29630.786818    -40424.793893
5      -31536.604702    -40256.902528
20     -24809.263250    -39865.781781
11     -24951.416893    -39850.505241
8      -32137.910114    -39175.185748
6      -30867.490870    -39041.805432
7      -30922.706087    -38990.155156
21     -20849.414904    -38855.346354
12     -21177.170944    -38830.793389
22     -23009.887940    -38688.567818
13     -23200.778883    -38601.332032
14     -26670.024548    -37906.786142
23     -26610.561623    -37881.476548
15     -24770.302427    -37035.665279
24     -24672.596148    -36938.274315
25     -24843.022976    -36928.343736
16     -24935.097772    -36919.631255
17     -2780

In [434]:
best_pipeline = grid_search.best_estimator_

In [435]:
prob_train = best_pipeline.predict(X_train)
prob_test = best_pipeline.predict(X_test)

In [436]:
from sklearn.metrics import *

print("Train:")
r2 = r2_score(Y_train, prob_train)
print(f"R2 score: {r2}")
rmse = mean_squared_error(Y_train, prob_train, squared=False)
print(f"RMSE score: {rmse}")
mae = mean_absolute_error(Y_train, prob_train)
print(f"MAE score: {mae}")

print()
print("Test: ")

r2 = r2_score(Y_test, prob_test)
print(f"R2 score: {r2}")
rmse = mean_squared_error(Y_test, prob_test, squared=False)
print(f"RMSE score: {rmse}")
mae = mean_absolute_error(Y_test, prob_test)
print(f"MAE score: {mae}")

Train:
R2 score: 0.8775337874883026
RMSE score: 27026.947558792122
MAE score: 16790.36948062146

Test: 
R2 score: 0.7786758648007447
RMSE score: 41202.28530678698
MAE score: 25852.649285770734


In [399]:
!pip install mlflow dagshub
import mlflow
import dagshub

dagshub.init(repo_owner='Givi-Modebadze', repo_name='my-first-repo', mlflow=True)

experiment = "my second experimnet"
run_name = "Decision Tree with cleaned features"
mlflow.set_experiment(experiment)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 39.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 74.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.1/260.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.0/684.0 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: dacite
    Found existing installation: dacite 1.9.2
    Uninstalling dacite-1.9.2:
  

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=091a1026-b327-411f-9512-32e3c185de58&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=4e74c45f84127a9c96c49a0d86d6d45588589fa9238d96ff4805b0ab2198020f




Output()

Accessing as Givi-Modebadze

Initialized MLflow to track repo "Givi-Modebadze/my-first-repo"

Repository Givi-Modebadze/my-first-repo initialized!

2025/04/09 18:21:15 INFO mlflow.tracking.fluent: Experiment with name 'my second experimnet' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/058f098621a44d7096faef4e4c751472', creation_time=1744222875012, experiment_id='3', last_update_time=1744222875012, lifecycle_stage='active', name='my second experimnet', tags={}>

In [400]:
X_train.shape

(1168, 75)

In [402]:
mlflow.start_run(run_name=run_name)
mlflow.log_param("model_type", "DecisionTreeRegressor")
mlflow.log_param("random_state", 42)
mlflow.log_param("N-features", 75)
mlflow.log_param("cv_folds", kfold.n_splits)

mlflow.log_metrics({
    'test_r2': r2,
    'test_rmse': rmse,
    'test_mae': mae
})

r2 = r2_score(Y_train, prob_train)
rmse = mean_squared_error(Y_train, prob_train, squared=False)
mae = mean_absolute_error(Y_train, prob_train)

mlflow.log_metrics({
    'train_r2': r2,
    'train_rmse': rmse,
    'train_mae': mae
})

mlflow.log_param("best_params", grid_search.best_params_)
mlflow.log_metric("best_cv_score", grid_search.best_score_)

mlflow.sklearn.log_model(best_pipeline, "DecisionTreeRegressor")

mlflow.end_run()

2025/04/09 18:27:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Decision Tree with cleaned features at: https://dagshub.com/Givi-Modebadze/my-first-repo.mlflow/#/experiments/3/runs/40ff03f6458e466d80038f7b49ac88af
🧪 View experiment at: https://dagshub.com/Givi-Modebadze/my-first-repo.mlflow/#/experiments/3
